# OpenAI ADK example connecting to Sap Generative AI Hub via LiteLLM

## [How OpenAI ADK works](https://openai.github.io/openai-agents-python/)

## Installation
%pip install "openai-agents[litellm]"

## Credentials for SAP Gen AI Hub
Get the service key from your SAP BTP tenant with AI subscription.

Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
AICORE_AUTH_URL="https://* * * .authentication.sap.hana.ondemand.com/oauth/token"
AICORE_CLIENT_ID=" *** "
AICORE_CLIENT_SECRET=" *** "
AICORE_RESOURCE_GROUP=" *** "
AICORE_BASE_URL="https://api.ai.***.cfapps.sap.hana.ondemand.com/
```

## Run the OpenAI with LiteLLM and SAP LLMs

In [ ]:
from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv

Load your credentials as environment variables that Litellm can use automatically.

In [ ]:
load_dotenv()
set_tracing_disabled(True) # Disable OPEN_API_KEY error

Define the agent tool.

In [ ]:
@function_tool
def get_weather(city: str):
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Define the agent with the model and the tool. Define the async function with agent run

In [ ]:
agent = Agent(
    name="Assistant",
    instructions="You are a helpful weather assistant. "
            "When the user asks you about a specific city, "
            "use the 'get_weather' tool to find the information about the weather. "
            "Answer with a TV weather report in two sentences, including a small joke.",
    model=LitellmModel(model="sap/gpt-4.1"),
    tools=[get_weather],
)


Run the function

In [ ]:
city = "london"
result = await Runner.run(agent, f"What's the weather in {city}?")
print(result.final_output)